In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', 'LM(타갈로그어)', '번역(한-타)', '번역(타-한)', '타갈로그어 번역 교정']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            text_id = line['id']
            source_language = 'ko-KR'
            target_language = 'tl-PH'
            source_text = line['source']
            preference_order = ['revision.revision2', 'revision.revision1', 'target']
            target_texts = []
            for pref in preference_order:
                keys = pref.split('.')
                text = line
                for key in keys:
                    text = text[key]
                text = str(text).replace('  ', ' ').strip()
                if text and text not in target_texts:
                    target_texts.append(text)
            
            #### data preprocess end 
            
            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            data['text'] = source_text
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end
            
            
            #### LM(타갈로그어)
            data = {'text': None}
            ## preprocess data from line
            data['text'] = target_texts[0]
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(타갈로그어) end
            
                        
            #### 번역(한-타)
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['source_text'] = source_text
            data['input']['source_language'] = source_language
            data['input']['target_language'] = target_language
            data['output'] = target_texts[0]
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(한-타) end
            
            
            #### 번역(타-한)
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['source_text'] = target_texts[0]
            data['input']['source_language'] = target_language
            data['input']['target_language'] = source_language
            data['output'] = source_text
            ## preprocess data from line end
            task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(타-한) end
            
            
            if len(target_texts) > 1:
                #### 타갈로그어 번역 교정
                ### 'output' is list of output sorted by relevance (1st is most relevant)
                data = {'input': {}, 'output': None}
                ## preprocess data from line
                data['input']['source_text'] = source_text
                data['input']['source_language'] = source_language
                data['input']['target_language'] = target_language
                data['output'] = target_texts
                ## preprocess data from line end
                task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 타갈로그어 번역 교정 end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train: 100%|██████████| 1/1 [00:03<00:00,  3.26s/it]
valid: 0it [00:00, ?it/s]
